# P03_06_Multivar_tf_supervised




DNN Supervised approach to forecast the ICME 18 ahead.

In [1]:
section_fig_main_title = 'P03_06_Multivar_tf_supervised'

In [7]:
import os
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))
print("PATH:", os.environ.get('PATH'))

PYTHONPATH: /usr/local/spark//python:/usr/local/spark//python:
PATH: /usr/local/spark/bin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/Users/amin/miniconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/opt/X11/bin:/Library/Apple/usr/bin


In [8]:
import os, sys
import requests
from urllib.parse import urljoin, urlparse
# from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
from pandas.plotting import lag_plot

from tensorflow import keras
from tensorflow.keras import layers


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "/Users/amin/Desktop/proj/env/bin/python"
  * The NumPy version is: "1.19.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


In [ ]:
!which python

In [5]:
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_absolute_error
# from xgboost import XGBRegressor
# !/Users/amin/Desktop/proj/env/bin/python -m pip install numpy

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error

In [ ]:
# !/Users/amin/anaconda3/bin/python -m pip install --upgrade tensorflow 
# pip install --upgrade tensorflow 

In [ ]:

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD

import math
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
from pandas.plotting import autocorrelation_plot
import gif

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
import gif
from tqdm import tqdm

In [ ]:
from sklearn import metrics
import statsmodels.api as sm

## Read Data 

Using the univariate evenly spaced time series data from P03_01.

In [ ]:
csv_path = "../Data/Data_ICME_Edited_Decoded_out_P02_03.csv"
dataset = pd.read_csv(csv_path, infer_datetime_format=True)

In [ ]:
dataset.index = dataset['ICME Plasma/Field Start Y/M/D (UT) (b)']

In [ ]:
dataset.head()

In [ ]:
dataset.columns

Following columns are used for multi-var time series modeling.

```BDE? (e)``` : Evidence of BiDirectional suprathermal Electron strahls (BDE) in ACE/SWEPAM Observations

```MC? (l)``` :  magnetic cloud has been reported in association with the ICME 

```V_ICME (km/s) (i)``` : ICME speed



In [ ]:
# keep_cols = ['BDE? (e)', 'Qual. (g)',
#        'dV (km/s) (h)', 'V_ICME (km/s) (i)', 'B (nT) (k)',
#        'MC? (l)', 'Dst (nT) (m)', 'Normalized_time_interval']

keep_cols = ['ICME Plasma/Field Start Y/M/D (UT) (b)' ,'BDE? (e)', 'V_ICME (km/s) (i)','MC? (l)']
dataset = dataset[keep_cols]
dataset.head()

In [ ]:
dataset.info()

## Make Data evenly spaced time series

In [ ]:
# fix the date dtype
dataset['ICME Plasma/Field Start Y/M/D (UT) (b)']= pd.to_datetime(dataset['ICME Plasma/Field Start Y/M/D (UT) (b)'])

In [ ]:
dataset = dataset.set_index('ICME Plasma/Field Start Y/M/D (UT) (b)').resample('18D').pad()

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
# remove first row
dataset = dataset.iloc[1: , :]
dataset.head()

## Preparing data 

Here I make new data set using pandas library. The shift method is used along with 60 lags. The goal is to use 60 lags (timestamps) to forecast the future timestamps (18 days ahead).

In [ ]:
dataset.shift(periods=1)

In [ ]:
cols = dataset.columns
cols[0]

In [ ]:
# df_pd

In [ ]:
lag=2

In [ ]:
df_pd = pd.DataFrame(dataset)
df_pd_shifted = pd.DataFrame(dataset)

cols = dataset.columns

for i in range(1,lag+1):

    
    shifted_df = df_pd.shift(periods=i)
    shifted_df.columns = [(cols[0] + 'lag' + str(i)), 
                          (cols[1] + 'lag' + str(i)), 
                          (cols[2] + 'lag' + str(i))] 
    
    df_pd_shifted = pd.concat([df_pd_shifted, shifted_df], axis=1)
    
    
    
    

# discard the first NaN rows 
df_pd_shifted = df_pd_shifted[lag:]



In [ ]:
for i in range(1,lag+1):
    
    for col in cols:
    
        colname = col + 'lag' + str(i) 

        df_pd_shifted[colname].astype('int32').dtypes

In [ ]:
df_pd_shifted.head()

In [ ]:
dataset = df_pd_shifted

## Split the data into training and test sets

#### https://www.tensorflow.org/tutorials/keras/regression

In [ ]:
# train_dataset = dataset.sample(frac=0.69, random_state=0)
# test_dataset = dataset.drop(train_dataset.index)

In [ ]:
split_frac= 0.8
num_train_rows = int(split_frac*len(dataset))
num_test_rows = len(dataset) - num_train_rows

In [ ]:
train_dataset, test_dataset = dataset[0:num_train_rows] , dataset[num_train_rows:]

In [ ]:
train_dataset

## Split features from labels

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('V_ICME (km/s) (i)')
test_labels = test_features.pop('V_ICME (km/s) (i)')

## Normalization

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

## The Normalization layer

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(train_features))

In [ ]:
print(normalizer.mean.numpy())

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
    
    print('First example:', first)
    print()
    print('Normalized:', normalizer(first).numpy())

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim(0,)
    plt.xlabel('Epoch')
    plt.ylabel('Error V_ICME (km/s) (i)')
    plt.legend()
    plt.grid(True)

## Regression using a DNN and multiple inputs

In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
      norm,
      layers.Dense(8, activation='relu'),
#       layers.Dense(32, activation='relu'),
      layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=2000)

In [ ]:
plot_loss(history)

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Error V_ICME (km/s) (i)]')
plt.ylabel('Predictions [Error V_ICME (km/s) (i)]')
lims = [200, 650]
plt.xlim(lims)
plt.ylim(lims)
# _ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Error V_ICME (km/s) (i)]')
_ = plt.ylabel('Count')

In [ ]:
len(train_labels)

In [ ]:
test_predictions

In [ ]:
test_labels

In [ ]:
yhat_ser = pd.Series(test_predictions)
yhat_ser.index = test_labels.index

In [ ]:
# train_features = train_dataset.copy()
# test_features = test_dataset.copy()

# train_labels = train_features.pop('V_ICME (km/s) (i)')
# test_labels = test_features.pop('V_ICME (km/s) (i)')

In [ ]:
len(train_labels)

In [ ]:
ahead = 153
past = 100


testy = test_labels
yhat = yhat_ser



plt.figure(figsize=(16,8))

plt.plot(train_labels[339-past:], 'r',label='history')
plt.vlines(train_labels[len(train_labels)-1:len(train_labels)].index,0, 700, colors='k', linestyles='dashed')

plt.plot(testy[0:ahead], 'r^',label='Test')
plt.plot(testy[0:ahead], 'r')
#     test.plot(kind='scatter')

plt.plot(yhat[0:ahead], color='blue',label='Predicted')
plt.title('DNN model', fontsize=10)
plt.xlabel('Date', fontsize=10)
plt.ylabel('ICME km/s', fontsize=10)


plt.xticks(fontsize=10)
plt.yticks(fontsize=10)


plt.legend(fontsize=10)
# plt.ylim(200,)

## rs2

In [ ]:
real_values = test_labels
predited_values = test_predictions

coefficient_of_dermination = r2_score(real_values, predited_values)
coefficient_of_dermination

rs2 = round(coefficient_of_dermination,2)
print(rs2)

## MAE

In [ ]:
mae = mean_absolute_error(real_values, predited_values)
mae = round(mae, 2)
mae

In [ ]:
plt.figure(figsize=(10,10))


real_values = test_labels
predited_values = test_predictions


plt.scatter(real_values, predited_values, color='r', 
            alpha=0.4, label=f'Validation Data with R2 = {rs2}',s=100)
plt.xlabel("Normalized Real ICME speed", fontsize=14)
plt.ylabel("Normalized Predited ICME speed", fontsize=14)
plt.xticks(fontsize=12), plt.yticks(fontsize=12)
plt.axline([0, 0], [1, 1], color ='k')
# plt.hlines(0, -4, 4, linestyles="--", color ='k')
# plt.vlines(0, -4, 4, linestyles="--", color ='k')

# lims = [300, 650]
# plt.xlim(lims)
# plt.ylim(lims)

plt.title("DNN model | supervised | ICME speed forcast results for 18 \
days ahead\nData from 18 days lags is used for prediction", fontsize=14)

plt.tight_layout()
plt.legend(fontsize=14)

plt.savefig("../Figures/" + section_fig_main_title + "DNN18daysAhead.png")

In [ ]:
dnn_model.save('../Models/dnn_model_tf_p03_06')

The results are not very impressive. The DNN does not encounter overfitting however applying small networks is not helpful. Applying split fraction 0.8 results in improved MAE values.
4 lags (4*18 days) information was used for prediction next time